# VacationPy
----

In [234]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from config import g_key

# Access maps with API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [235]:
# File to Load
file_to_load = "Resources/cities.csv"

# Read file and store into a data frame
cities_df = pd.read_csv(file_to_load)
cities_df.head(10)

,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,New Norfolk,77,AU,1602181378,93,-42.78,147.06,52.00,1.21
1,1,Geraldton,1,AU,1602181378,93,-28.77,114.60,53.60,8.05
2,2,Illapel,2,CL,1602181378,29,-31.63,-71.17,75.87,9.31
3,3,Ekangala,0,ZA,1602181378,56,-25.70,28.75,69.80,5.82
4,4,Busselton,88,AU,1602181378,73,-33.65,115.33,52.00,5.53
5,5,Kapaa,40,US,1602181132,74,22.08,-159.32,80.60,6.93
6,6,Camacha,40,PT,1602181378,73,33.08,-16.33,73.40,6.93
7,7,Meadow Lake,1,US,1602181379,13,34.80,-106.54,77.00,5.44
8,8,Punta Arenas,40,CL,1602181379,50,-53.15,-70.92,53.60,34.45
9,9,Beloha,0,MG,1602181379,81,-25.17,45.05,64.13,7.49


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [236]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Latitude", "Longitude"]].astype(float)

# Convert humidity to float and store
humidity_map = cities_df["Humidity"].astype(float)

In [244]:
# Create a humidity Heatmap layer
fig = gmaps.figure(center=(0, 0), zoom_level=2.25)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_map, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [216]:
# Locating cities with nice weather conditions
nice_weather_df = cities_df.loc[(cities_df['Max Temperature'] > 64) & (cities_df['Max Temperature'] < 86) 
                                & (cities_df['Humidity'] < 66) & (cities_df['Cloudiness'] == 0),:]

del nice_weather_df['Unnamed: 0']

# Checking to see if data meets conditions
nice_weather_df.head(20)

,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
3,Ekangala,0,ZA,1602181378,56,-25.70,28.75,69.80,5.82
32,Guerrero Negro,0,MX,1602181288,50,27.98,-114.06,80.38,15.43
42,Port Hedland,0,AU,1602181385,55,-20.32,118.57,64.40,5.82
78,Tiznit Province,0,MA,1602181392,40,29.58,-9.50,76.60,4.70
142,Mount Isa,0,AU,1602181404,24,-20.73,139.50,69.80,13.87
163,Fatehpur,0,IN,1602181407,46,25.93,80.80,78.31,1.77
218,Mizdah,0,LY,1602181418,58,31.45,12.98,70.81,14.27
246,Sur,0,OM,1602181425,55,22.57,59.53,79.68,8.88
264,Bosaso,0,SO,1602181427,54,11.28,49.18,82.06,4.05
279,Borūjerd,0,IR,1602181430,14,33.90,48.75,78.80,2.24


In [217]:
# Count of cities returned
nice_weather_df.count()

City Name          17
Cloudiness         17
Country            17
Date               17
Humidity           17
Latitude           17
Longitude          17
Max Temperature    17
Wind Speed         17
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [212]:
hotel_df = nice_weather_df.loc[:,['City Name', 'Country', 'Latitude', 'Longitude']]
hotel_df['Hotel Name'] = ""
hotel_df.head(17)

,City Name,Country,Latitude,Longitude,Hotel Name
3,Ekangala,ZA,-25.70,28.75,
32,Guerrero Negro,MX,27.98,-114.06,
42,Port Hedland,AU,-20.32,118.57,
78,Tiznit Province,MA,29.58,-9.50,
142,Mount Isa,AU,-20.73,139.50,
163,Fatehpur,IN,25.93,80.80,
218,Mizdah,LY,31.45,12.98,
246,Sur,OM,22.57,59.53,
264,Bosaso,SO,11.28,49.18,
279,Borūjerd,IR,33.90,48.75,


In [221]:
# Setting up API request for hotels in each city

for index, row in hotel_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            "type": "lodging",
            "keyword": "hotel",
            "radius": 10000,
            "key": g_key,
        }

        # Getting variables of interest
        city = row['City Name']
        lat = row['Latitude']
        lon = row['Longitude']
        params['location'] = f"{lat}, {lon}"
        
        # Assemble url and make API request
        print(f"Retrieving Results for {city}.")
        response = requests.get(base_url, params=params).json()
        results = response['results']
        print(f"A hotel in {city} was found. Hotel Name: {results[0]['name']}.")
        
        # Add to empty column in DF
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
                           
    except:
        print(f"No hotels found in {city}.")
        
print("------------")
print("Printing complete.")

Retrieving Results for Ekangala.
A hotel in Ekangala was found. Hotel Name: Chukuru Lodge.
Retrieving Results for Guerrero Negro.
A hotel in Guerrero Negro was found. Hotel Name: Hotel Los Caracoles.
Retrieving Results for Port Hedland.
A hotel in Port Hedland was found. Hotel Name: The Esplanade Hotel.
Retrieving Results for Tiznit Province.
No hotels found in Tiznit Province.
Retrieving Results for Mount Isa.
A hotel in Mount Isa was found. Hotel Name: ibis Styles Mt Isa Verona.
Retrieving Results for Fatehpur.
A hotel in Fatehpur was found. Hotel Name: Era Garden.
Retrieving Results for Mizdah.
A hotel in Mizdah was found. Hotel Name: فندق بنيه تحتيه.
Retrieving Results for Sur.
A hotel in Sur was found. Hotel Name: Sur Plaza Hotel.
Retrieving Results for Bosaso.
A hotel in Bosaso was found. Hotel Name: Gacayte Hotel.
Retrieving Results for Borūjerd.
A hotel in Borūjerd was found. Hotel Name: Hotel khatam.
Retrieving Results for Waddān.
No hotels found in Waddān.
Retrieving Results 

In [222]:
# Dataframe with hotel names added
hotel_df.head(17)

,City Name,Country,Latitude,Longitude,Hotel Name
3,Ekangala,ZA,-25.70,28.75,Chukuru Lodge
32,Guerrero Negro,MX,27.98,-114.06,Hotel Los Caracoles
42,Port Hedland,AU,-20.32,118.57,The Esplanade Hotel
78,Tiznit Province,MA,29.58,-9.50,AMICAL TITRIT OUGLO ودادية تيتريت أوكلو
142,Mount Isa,AU,-20.73,139.50,ibis Styles Mt Isa Verona
163,Fatehpur,IN,25.93,80.80,Era Garden
218,Mizdah,LY,31.45,12.98,فندق بنيه تحتيه
246,Sur,OM,22.57,59.53,Sur Plaza Hotel
264,Bosaso,SO,11.28,49.18,Gacayte Hotel
279,Borūjerd,IR,33.90,48.75,Hotel khatam


In [224]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [232]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure(center=(0, 0), zoom_level=2)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))